### Early Stopping

This notebook tests the effectiveness of early stopping criterion, and re-runs trials to test more thoroughly. IQL early stopping criteria is used instead of the TD3+BC one if 'iql' is in the name.

In [ ]:
import os
import sys
import json
import numpy as np
import pandas as pd
from decouple import config
MAIN_PATH = config('MAIN_PATH')
sys.path.insert(1, MAIN_PATH)
from visualiser.core import ExperimentVisualise, plot_testing_rewards, plot_training_logs, plot_value_function
from visualiser.core import plot_episode, plot_training_action_summary, plot_training_logs
from visualiser.core import display_commands, plot_testing_metric, display_commands_v2#, plot_testing_average_metric
from metrics.metrics import time_in_range
from metrics.statistics import calc_stats
from agents.models.actor_critic_td3_bc import ActorCritic as TD3BC_ActorCritic
from agents.models.actor_critic_iql import ActorCritic as IQL_ActorCritic
import numpy.polynomial.polynomial as poly 
from utils.worker import OnPolicyWorker, OffPolicyWorker, OfflineSampler
from omegaconf import DictConfig, OmegaConf
import matplotlib.gridspec
from matplotlib import pyplot as plt
import math
import torch
from utils.logger import setup_folders, Logger


%matplotlib inline 

EXP_NAME = "offline_iql_t3_6_0"

LABEL = EXP_NAME

# custom functions


PI_GRAD_THRESHOLD = 140
PI_CONCAV_THRESHOLD = 0.00008
PI_CONCAV_CONVERGENCE_THRESHOLD = 0.0001
PI_CONCAV_CONVERGENCE_N = 3
MONOTONIC_EPISLON = 1e-5

TITLE_FONT_SIZE = 24
AXIS_FONT_SIZE = 16

TAKE_BEST = True
SHOW_GRAPHS = False

NAME_PREFIX = "offline_td3_purebc"
USE_IQL_CRITERIA = 'iql' in NAME_PREFIX.lower()

def limit(n,t,b): return max(min(n,t),b)

def check_end_condition_total(
        val_grads, val_losses, pi_grads, steps,
        debug_show=False
    ):
    
    # default to last network
    epochs = len(val_grads)
    total_ind = epochs



    pi_grad_grads = []
    for n in range(epochs-1):
        dy = pi_grads[n+1] - pi_grads[n]
        dx = steps[n+1] - steps[n]
        pi_grad_grads.append( (dy / dx) )

    if debug_show: 
        print(pi_grads)
        print(pi_grad_grads)
    
    rough_monotonic = all([pi_grad_grad < MONOTONIC_EPISLON for pi_grad_grad in pi_grad_grads])
    if not rough_monotonic and debug_show: print("WARNING! Not monotonic")
    

    # calculated based on pi grad threshold
    for n, pi_grad in enumerate(pi_grads):
        if abs(pi_grad) < PI_GRAD_THRESHOLD:
            total_ind = (n - 1)
            break

    total_ind = min(max(total_ind, 0), epochs -1)

    return total_ind

def check_end_condition_total_alt(
        val_grads, val_losses, pi_grads, steps,
        debug_show=False
    ):
    
    # default to last network
    epochs = len(val_grads)
    total_ind = epochs



    pi_grad_grads = []
    for n in range(epochs-1):
        dy = pi_grads[n+1] - pi_grads[n]
        dx = steps[n+1] - steps[n]
        pi_grad_grads.append( (dy / dx) )

    if debug_show: 
        print(pi_grads)
        print(pi_grad_grads)
    

    ## maximum pi grad
    max_grad_ind = max(list(range(epochs-1)), key= lambda ind : pi_grads[ind])
    total_ind = max_grad_ind
        


    total_ind = min(max(total_ind, 0), epochs -1)

    return total_ind

def create_file_paths(path, seeds, filename, FILES):
    for seed in seeds:
        FILES.append(path + filename + str(seed)+'.csv')
    return FILES
    
def generate_reward_list(exp_name):
    exp1 = ExperimentVisualise(id=exp_name, version=1.1, plot_version=1, test_seeds=5000)

    disp_arr = display_commands_v2(command)
    
    exp_vis = {
                '1':{ "id":exp1, "color":'r',"show": disp_arr[1], "label":exp_name},
    }

    metric = 'reward'

    path, seeds, filename = exp_vis['1']['id'].get_file_paths()
    FILES = []
    FILES = create_file_paths(path, seeds, filename, FILES)
    cur_length, full_arr, refined = [], [], []
    for file in FILES:
        reward_summary = pd.read_csv(file)
        cur_length.append(reward_summary.shape[0])
        full_arr.append(reward_summary[metric])
    for x in full_arr:
        refined.append(x[0:min(cur_length)])
    data = pd.concat(refined, axis=1)
    data['mean'] = data.mean(axis=1)
    
    return list(data['mean'])
    
def generate_end_index(exp_name, return_proportion=False, debug_show=False,use_alt=False):
    df = pd.read_csv(MAIN_PATH + '/' + 'results/' + exp_name + '/experiment_summary.csv', header="infer")
    df_len = len(df["value_grad"])
    
    exp = ExperimentVisualise(id=exp_name, version=1.1, plot_version=1, test_seeds=5000)
    d1 = exp.get_training_logs()

    steps = list(d1['steps'])
    val_grad = list(d1['value_grad'])
    val_loss = list(d1['val_loss'])
    pi_grad = list(d1['policy_grad'])

    if use_alt: ind = check_end_condition_total_alt(val_grad, val_loss, pi_grad, steps, debug_show=debug_show)
    else: ind = check_end_condition_total(val_grad, val_loss, pi_grad, steps, debug_show=debug_show)
    if return_proportion: return ind / (df_len - 1)
    return ind

def take_rows(exp_name, offset_ind=5000, workers=5, episode=0):
    exp_folder = MAIN_PATH + '/results/' + exp_name + '/' 

    dfs = []
    files = [exp_folder + f'testing/worker_episode_summary_{offset_ind + n}.csv' for n in range(workers)]
    for file in files:
        df = pd.read_csv(file)
        df["full_length"] = (df['t'] == 288).astype(int)
        df["adj_normo"] = (df["t"] / 288) * df["normo"]
        # print(df["adj_normo"][:5])
        # print(df["t"][:5])
        # print(df["normo"][:5])
        if episode != None:
            df_filt = df[df['epi'] == episode]
            dfs.append(df_filt)
        else:
            dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

def get_mn_sd(li):
    return (np.mean(li), np.std(li))

def find_best_tir_index(exp_name, bound, workers=5): #bound exclusive
    ep_dfs = []
    for epi in range(bound):
        ep_dfs.append(take_rows(exp_name, episode=epi+1, workers=workers))

    return max(list(range(bound)), key = lambda ind: get_mn_sd(ep_dfs[ind]["adj_normo"])[0])

def find_best_reward_index(exp_name, bound, workers=5): #bound exclusive
    ep_dfs = []
    for epi in range(bound):
        ep_dfs.append(take_rows(exp_name, episode=epi+1, workers=workers))

    return max(list(range(bound)), key = lambda ind: get_mn_sd(ep_dfs[ind]["reward"])[0])

# modified graphing functions

def plot_testing_average_metric(dict, groups, type, dis_len, metric, goal, fill,vline=None, title=None):
    fig = plt.figure(figsize=(16,9))
    ax = fig.add_subplot(111)

    for groupings in range(0, len(groups)):
        FILES = []
        for i in groups[groupings]:  # exp's inside the group
            # will give the exp list
            path, seeds, filename = dict[i]['id'].get_file_paths()
            FILES = create_file_paths(path, seeds, filename, FILES)
        cur_length, full_arr, refined = [], [], []
        for file in FILES:
            reward_summary = pd.read_csv(file)
            cur_length.append(reward_summary.shape[0])
            full_arr.append(reward_summary[metric])
        for x in full_arr:
            refined.append(x[0:min(cur_length)])
        data = pd.concat(refined, axis=1)
        data['mean'] = data.mean(axis=1)

        if type == 'normal':
            data['std_dev'] = data.std(axis=1)
            data['max'] = data['mean'] + data['std_dev']  # * 2
            data['min'] = data['mean'] - data['std_dev']  # * 2
        else:
            data['max'] = data.max(axis=1)
            data['min'] = data.min(axis=1)

        data['steps'] = np.arange(len(data))
        data['steps'] = (data['steps'] + 1) * dict[i]['id'].training_workers * dict[i]['id'].args['agent']['n_step']

        ax.plot(data['steps'], data['mean'], color=dict[i]['color'], label=dict[i]['label'])
        if fill:
            ax.fill_between(data['steps'], data['min'], data['max'], color=dict[i]['color'], alpha=0.1)

    ax.axhline(y=goal, color='k', linestyle='--')
    
    start_step = list(data['steps'])[0]
    end_step = list(data['steps'])[-1]
    if vline != None:
        ax.axvline(x=start_step + vline*(end_step - start_step), color='b', linestyle='-')

    graph_title =  title if title is not None else 'Training Curve'
    # ax.set_title(graph_title, fontsize=32)
    # ax.legend(loc="upper left", fontsize=16)
    ax.set_ylabel('Average Validation Reward', fontsize=TITLE_FONT_SIZE) #ax.set_ylabel(metric)
    ax.set_xlabel('Training Interactions', fontsize=TITLE_FONT_SIZE)
    ax.grid()
    ax.set_xticks([])
    ax.set_xlim(0, dis_len)
    ax.set_ylim(0, 320)
    ax.tick_params(axis='y', which='major', labelsize=AXIS_FONT_SIZE) 
    plt.show()

def plot_training_logs(mode, exp_dict, dis_len, params,vline=None, cols=1,val_grad_poly=None):
    fig = plt.figure(figsize=(16, 9))
    subplots = []
    tot_plots = len(params)

    for i in range(0, tot_plots):
        subplots.append(fig.add_subplot(math.ceil(tot_plots/cols), cols, i+1))
        subplots[i].grid(True)
        subplots[i].set_xlim(0, dis_len)
        subplots[i].set_title( "" )#params[i])
        subplots[i].set_xlabel( "Interactions", fontsize=TITLE_FONT_SIZE )#params[i])
        subplots[i].set_ylabel( "Summed Policy Gradient", fontsize=TITLE_FONT_SIZE )#params[i])
        subplots[i].set_xticks([])
        subplots[i].tick_params(axis='y', which='major', labelsize=AXIS_FONT_SIZE) 

    for exp in exp_dict:
        if exp_dict[exp]['show']:
            if mode == 'ppo':
                # ['exp_var', 'true_var','val_loss', 'policy_grad', 'value_grad', 'pi_loss', 'avg_rew']
                d1 = exp_dict[exp]['id'].get_training_logs()
            elif mode == 'aux':
                # ['pi_aux_loss', 'vf_aux_loss', 'pi_aux_grad', 'vf_aux_grad']
                d1 = exp_dict[exp]['id'].get_aux_training_logs()
            elif mode == 'planning':
                # ['plan_grad', 'plan_loss']
                d1 = exp_dict[exp]['id'].get_planning_training_logs()
            else:
                d1 = 0
                print('Invalid mode selected')
                exit()
            for j in range(0, tot_plots):
                start_step = list(d1['steps'])[0]
                end_step = list(d1['steps'])[-1]

                if params[j] == 'value_grad' and val_grad_poly != None:
                    polyline = np.linspace(start_step, end_step)
                    subplots[j].plot(polyline, val_grad_poly(polyline))
                subplots[j].plot(d1['steps'], d1[params[j]], color=exp_dict[exp]['color'], label=exp_dict[exp]['id'].id)
                subplots[j].axvline(x=start_step + vline*(end_step - start_step), color='b', linestyle='-')

    # for i in range(0, tot_plots):
    #     subplots[i].legend(loc="upper right")
    plt.show()

def calculate_r_squared(y_data, RSS):
    y_mean = np.mean(y_data)
    TSS = sum([(y_mean - y)**2 for y in y_data])
    return 1 - (RSS/TSS)

def generate_poly(steps, val_grad):
    fitted_poly = poly.Polynomial.fit(steps, val_grad, 2, full=True) #fit quadratic model
    r_2 = calculate_r_squared(val_grad, fitted_poly[1][0][0])
    return r_2, fitted_poly[0]

# main

command = []
# dis_len = 1.7 * 100000 #BC
# dis_len = 2.5 * 1000000 #TD3+BC
dis_len = 3.5 * 100000 #IQL

disp_arr = display_commands_v2(command)

exp1 = ExperimentVisualise(id=EXP_NAME, version=1.1, plot_version=1, test_seeds=5000)

exp_vis = { '1':{ "id":exp1, "color":'r',"show": disp_arr[1], "label":LABEL}}

exp_logs = exp_vis['1']['id'].get_training_logs()
goodness_val_grad_fit, val_grad_poly = generate_poly(list(exp_logs['steps']), list(exp_logs['value_grad']))

df = pd.read_csv(MAIN_PATH + '/' + 'results/' + EXP_NAME + '/experiment_summary.csv', header="infer")
df_len = len(df["value_grad"])

if TAKE_BEST:
    chosen_ind = find_best_reward_index(EXP_NAME,df_len,workers=4)
    chosen_ind_prop = (chosen_ind) / (df_len - 1)
else:
    chosen_ind_prop = generate_end_index(EXP_NAME, True, debug_show=True, use_alt=USE_IQL_CRITERIA)
    chosen_ind = generate_end_index(EXP_NAME, use_alt=USE_IQL_CRITERIA)
    print("\tTerminated on index",chosen_ind)






exps = []
for patient_id in list(range(0,10)) + list(range(20,30)):
    for seed in range(3):
        exps.append( f"{NAME_PREFIX}_{patient_id}_{seed}" )

percents = []
for exp_name in exps:
    
    print(f"{exp_name}: ")
    reward_list = generate_reward_list(exp_name)
    chosen_ind = generate_end_index(exp_name,debug_show=False, use_alt=False)
    opt_ind = find_best_tir_index(exp_name, bound=len(reward_list), workers=4)

    percent = 100 * reward_list[chosen_ind] / max(reward_list)

    print(f"\tReward obtained is {reward_list[chosen_ind]:.2f}/{max(reward_list):.2f}, {percent:.2f}%")
    print()

    percents.append(percent)


print(f"Average returns of {np.mean(percents):.2f}+-{np.std(percents):.2f}%")




C:\Users\Crago\AppData\Local\Temp\ipykernel_23028\1443756178.py:306: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - (RSS/TSS)


offline_td3_purebc_0_0: 
	Reward obtained is 283.82/283.82, 100.00%

offline_td3_purebc_0_1: 
	Reward obtained is 87.70/283.09, 30.98%

offline_td3_purebc_0_2: 
	Reward obtained is 280.06/280.61, 99.81%

offline_td3_purebc_1_0: 
	Reward obtained is 110.68/230.34, 48.05%

offline_td3_purebc_1_1: 
	Reward obtained is 221.64/234.10, 94.68%

offline_td3_purebc_1_2: 
	Reward obtained is 103.77/245.18, 42.32%

offline_td3_purebc_2_0: 
	Reward obtained is 96.08/247.57, 38.81%

offline_td3_purebc_2_1: 
	Reward obtained is 261.05/261.05, 100.00%

offline_td3_purebc_2_2: 
	Reward obtained is 46.24/238.83, 19.36%

offline_td3_purebc_3_0: 
	Reward obtained is 242.87/242.87, 100.00%

offline_td3_purebc_3_1: 
	Reward obtained is 191.39/234.61, 81.58%

offline_td3_purebc_3_2: 
	Reward obtained is 37.17/244.92, 15.18%

offline_td3_purebc_4_0: 
	Reward obtained is 200.52/200.52, 100.00%

offline_td3_purebc_4_1: 
	Reward obtained is 237.89/256.77, 92.64%

offline_td3_purebc_4_2: 
	Reward obtained is 256

In [7]:
""" 
5000 - Validation during training
8000 - early stopping
9000 - picking best adj normoglycemia (TIR)
10000 - picking best reward
"""

import datetime
from pathlib import Path

sys.path.insert(0, MAIN_PATH + '/experiments/')

TRIAL_N = 50
VLD_OFFSET = 9000
IQL_EARLY_STOPPING = True

def open_arg_file(file_dest):
    with open(file_dest,'r') as fp:
        args_dict = json.load(fp)
        fp.close()
    return OmegaConf.create(args_dict)

def rec_alter(tdict, replacee, replacer):
    for k in tdict:
        if type(tdict[k]) == type(dict()):
            tdict[k] = rec_alter(tdict[k], replacee, replacer)
        else:
            if type(tdict[k]) == type(""): 
                tdict[k] = tdict[k].replace(replacee, replacer)
    return tdict

def zombify_arg_file(file_dest):
    with open(file_dest,'r') as fp:
        args_dict = json.load(fp)
        fp.close()
    
    nest_keys = []
    for k in args_dict.keys():
        if type(args_dict[k]) == type(dict()):
            nest_keys.append(k)
    
    args_dict["agent"]["validation_agent_id_offset"] = VLD_OFFSET
    args_dict["agent"]["n_val_trials"] = TRIAL_N
    args_dict["agent"]["n_testing_workers"] = 0
    args_dict["agent"]["n_training_workers"] = 0
    args_dict["logger"]["experiment_logs"] = []

    for k in nest_keys:
        for n_k in args_dict[k].keys():
            if not n_k in args_dict: args_dict[n_k] = (args_dict[k])[n_k]
    
    rec_alter(args_dict, f"../results/", MAIN_PATH + '/results/')

    

    
    return OmegaConf.create(args_dict)

def delete_files_with_prefix(folder_path, prefix1="worker_episode_8", prefix2="worker_episode_summary_8", show=False) -> None:
    """
    Delete all files in the given folder whose names start with the given prefix.

    Args:
        folder_path (str): Path to the folder containing files.
        prefix (str): The string prefix to match filenames against.
    """
    folder = Path(folder_path)

    if not folder.is_dir():
        raise ValueError(f"{folder_path} is not a valid directory.")

    for file in folder.iterdir():
        if file.is_file() and (file.name.startswith(prefix1) or file.name.startswith(prefix2)):
            try:
                file.unlink()  # Deletes the file
                if show: print(f"Deleted: {file}")
            except Exception as e:
                print(f"Failed to delete {file}: {e}")

def run_validation_trials(exp_name, checkpoint, use_iql=False):
    exp_folder = MAIN_PATH + '/results/' + exp_name + '/' 

    
    # args = open_arg_file(exp_folder + 'args.json')
    args = zombify_arg_file(exp_folder + 'args.json')
    args.n_action = args.n_actions

    print("patient_id", args.patient_id)

    
    if use_iql: #IQL 
        actor_path = exp_folder + f'checkpoints/episode_{max(0,checkpoint)}_policy_net.pth'
        critic_path = exp_folder + f'checkpoints/episode_{max(0,checkpoint)}_critic_net.pth'
        value_path = exp_folder + f'checkpoints/none.pth'

        # actor_path = exp_folder + f'checkpoints/episode_{max(0,checkpoint)}_policy_net.pth'
        # critic_path = exp_folder + f'checkpoints/episode_{max(0,checkpoint)}_critic_net.pth'
        # value_path = exp_folder + f'checkpoints/episode_{max(0,checkpoint)}_value_net.pth'

        policy = IQL_ActorCritic(args, load=True, actor_path=actor_path, critic_path=critic_path, value_path=value_path, device=args.agent.device).to(args.device) #TD3+BC ONLY
    else: #TD3+BC 
        actor_path = exp_folder + f'checkpoints/episode_{max(0,checkpoint)}_policy_net.pth'
        critic_path = exp_folder + f'checkpoints/episode_{max(0,checkpoint)}_value_net.pth'
        policy = TD3BC_ActorCritic(args, load=True, actor_path=actor_path, critic_path=critic_path, device=args.agent.device).to(args.device) #TD3+BC ONLY

    logger = Logger(args)
    validation_agents = [OnPolicyWorker(args=args, env_args=args, mode='testing', worker_id=i + args.validation_agent_id_offset) for i in range(args.n_val_trials)]
    
    with torch.no_grad():
        for i in range(args.agent.n_val_trials):
            validation_agents[i].rollout(policy=policy, buffer=None, logger=logger.logWorker)
    
     # calculate the final metrics.
    cohort_res, summary_stats = [], []
    secondary_columns = ['epi', 't', 'reward', 'normo', 'hypo', 'sev_hypo', 'hyper', 'lgbi',
                    'hgbi', 'ri', 'sev_hyper', 'aBGP_rmse', 'cBGP_rmse']
    data = []
    FOLDER_PATH = args.experiment_folder+'/testing/'
    for i in range(0, args.n_val_trials):
        test_i = 'worker_episode_'+str(args.validation_agent_id_offset+i)+'.csv'
        df = pd.read_csv(FOLDER_PATH+ '/'+test_i, dtype=float)
        normo, hypo, sev_hypo, hyper, lgbi, hgbi, ri, sev_hyper = time_in_range(df['cgm'])
        reward_val = df['rew'].sum()*(100/288)
        e = [[i, df.shape[0], reward_val, normo, hypo, sev_hypo, hyper, lgbi, hgbi, ri, sev_hyper, 0, 0]]
        dataframe = pd.DataFrame(e, columns=secondary_columns)
        data.append(dataframe)
    res = pd.concat(data)
    res['PatientID'] = args.patient_id
    res.rename(columns={'sev_hypo':'S_hypo', 'sev_hyper':'S_hyper'}, inplace=True)
    summary_stats.append(res)
    metric=['mean', 'std', 'min', 'max']
    print(calc_stats(res, metric=metric, sim_len=288))

    print('\nAlgorithm Training/Validation Completed Successfully.')
    print('---------------------------------------------------------')
    exit()

exp_names = exps

for exp_name in exp_names:
    exp_folder = MAIN_PATH + '/results/' + exp_name + '/' 
    patient_id = int(exp_name.split('_')[-2])
    seed = int(exp_name.split('_')[-1])

    print(f"========== {exp_name}: ")
    reward_list = generate_reward_list(exp_name)

    # chosen_ind = generate_end_index(exp_name,debug_show=True,use_alt=IQL_EARLY_STOPPING)
    chosen_ind = find_best_reward_index(exp_name, bound=len(reward_list), workers=4) 
    # chosen_ind_alt = find_best_tir_index(exp_name, bound=len(reward_list), workers=4) 

    # print(exp_name, chosen_ind, chosen_ind_alt, chosen_ind_alt==chosen_ind)
    percent = 100 * reward_list[chosen_ind] / max(reward_list)

    print(f"\tReward obtained is {reward_list[chosen_ind]:.2f}/{max(reward_list):.2f}, {percent:.2f}%, on index {chosen_ind}/{len(reward_list)}")

    # early_stopping_policy_dest = exp_folder + f'checkpoints/episode_{chosen_ind}_policy_net.pth'

    use_iql = 'iql' in exp_name
    delete_files_with_prefix(exp_folder + 'testing/', prefix1="worker_episode_" + str(VLD_OFFSET)[0], prefix2="worker_episode_summary_" + str(VLD_OFFSET)[0], show=False)
    run_validation_trials(exp_name, chosen_ind, use_iql=use_iql)
    print("New validation trials ran.")




========== offline_td3_purebc_0_0: 
	Reward obtained is 283.82/283.82, 100.00%, on index 7/16
patient_id 0
['adolescent#001', 'adolescent#002', 'adolescent#003', 'adolescent#004', 'adolescent#005', 'adolescent#006', 'adolescent#007', 'adolescent#008', 'adolescent#009', 'adolescent#010', 'child#001', 'child#002', 'child#003', 'child#004', 'child#005', 'child#006', 'child#007', 'child#008', 'child#009', 'child#010', 'adult#001', 'adult#002', 'adult#003', 'adult#004', 'adult#005', 'adult#006', 'adult#007', 'adult#008', 'adult#009', 'adult#010']


c:\Users\Crago\AppData\Local\Programs\Python\Python312\Lib\site-packages\gym\envs\registration.py:17: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


['adolescent#001', 'adolescent#002', 'adolescent#003', 'adolescent#004', 'adolescent#005', 'adolescent#006', 'adolescent#007', 'adolescent#008', 'adolescent#009', 'adolescent#010', 'child#001', 'child#002', 'child#003', 'child#004', 'child#005', 'child#006', 'child#007', 'child#008', 'child#009', 'child#010', 'adult#001', 'adult#002', 'adult#003', 'adult#004', 'adult#005', 'adult#006', 'adult#007', 'adult#008', 'adult#009', 'adult#010']


c:\Users\Crago\AppData\Local\Programs\Python\Python312\Lib\site-packages\gym\envs\registration.py:17: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


['adolescent#001', 'adolescent#002', 'adolescent#003', 'adolescent#004', 'adolescent#005', 'adolescent#006', 'adolescent#007', 'adolescent#008', 'adolescent#009', 'adolescent#010', 'child#001', 'child#002', 'child#003', 'child#004', 'child#005', 'child#006', 'child#007', 'child#008', 'child#009', 'child#010', 'adult#001', 'adult#002', 'adult#003', 'adult#004', 'adult#005', 'adult#006', 'adult#007', 'adult#008', 'adult#009', 'adult#010']


c:\Users\Crago\AppData\Local\Programs\Python\Python312\Lib\site-packages\gym\envs\registration.py:17: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


['adolescent#001', 'adolescent#002', 'adolescent#003', 'adolescent#004', 'adolescent#005', 'adolescent#006', 'adolescent#007', 'adolescent#008', 'adolescent#009', 'adolescent#010', 'child#001', 'child#002', 'child#003', 'child#004', 'child#005', 'child#006', 'child#007', 'child#008', 'child#009', 'child#010', 'adult#001', 'adult#002', 'adult#003', 'adult#004', 'adult#005', 'adult#006', 'adult#007', 'adult#008', 'adult#009', 'adult#010']


c:\Users\Crago\AppData\Local\Programs\Python\Python312\Lib\site-packages\gym\envs\registration.py:17: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


KeyboardInterrupt: 